In [3]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [4]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT * 
FROM Z3DMC.MEB_DGM""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY')
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY DMC ORDER BY ID_DMC DESC) AS rn
  FROM
    Z3DMC.MEB_DMC t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
"""
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [5]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [6]:
print(list(data.keys()))
# Useless tables
# MEB_CONTAINERS, MEB_DMC_GRADE, MEB_KS_WZORC

['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS']


In [7]:
# ONI CIRCUITS
# print(data['ONI_CIRCUITS'].head())
data['ONI_CIRCUITS'].drop(columns=['timestamp'], inplace=True)
print("\n ONI_CIRCUITS after drop of columns: \n")
print(data['ONI_CIRCUITS'].head())

KeyError: 'ONI_CIRCUITS'

In [8]:
# MEB_DGM
# print(data['MEB_DGM'].columns)
# print(data['MEB_DGM'].head(5))
# ('metal_level', 'metal_pressure') - We do not collect this data yet
data['MEB_DGM'].drop(columns=['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'], inplace=True)
print("\n MEB_DGM after drop of columns: \n")
print(data['MEB_DGM'].head(5))


 MEB_DGM after drop of columns: 

     id                    dmc  nr_dgm status  czas_fazy_1  czas_fazy_2  \
0   220  190627091611011003610    10.0      4       1850.0         88.0   
1  3416  190719071611011006077    10.0      1       2197.0         89.0   
2  3417  190719071811011006078    10.0      1       2184.0         86.0   
3  3418  190719072011011006079    10.0      1       2159.0         89.0   
4  3419  190719072111011006080    10.0      1       2175.0         89.0   

   czas_fazy_3  max_predkosc  cisnienie_tloka  cisnienie_koncowe  ...  \
0         47.5          6.52              7.0              264.0  ...   
1         46.5          6.59             11.0              285.0  ...   
2         45.0          6.52              8.0              284.0  ...   
3         45.5          6.54              6.0              284.0  ...   
4         47.0          6.57              8.0              283.0  ...   

   oni_temp_curr_f2  oni_temp_fore_f1  oni_temp_fore_f2  vds_air_pressure  

In [ ]:
#MEB_DMC
# print(data['MEB_DMC'].columns)
# print(data['MEB_DMC'].head(5))
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step'], inplace = True)
print("\n MEB_DMC after drop of columns: \n")
print(data['MEB_DMC'].head(5))

In [ ]:
#MEB_KO
# print(data['MEB_KO'].columns)
# print(data['MEB_KO'].head(5))
data['MEB_KO'].drop(columns = ['id_ko', 'data', 'timestamp', 'eks'], inplace = True)
print("\n MEB_KO after drop of columns: \n")
print(data['MEB_KO'].head(5))

In [ ]:
#MEB_KO_DGM
# print(data['MEB_KO_DGM'].columns)
# print(data['MEB_KO_DGM'].head(5))
data['MEB_KO_DGM'].drop(columns = ['id_ko','data_odlania', 'timestamp', 'operator'], inplace = True)
print("\n MEB_KO_DGM after drop of columns: \n")
print(data['MEB_KO_DGM'].head(5))

In [ ]:
# MEB_KS
# print(data['MEB_KS'].columns)
# print(data['MEB_KS'].head(5))
data['MEB_KS'].drop(columns = ['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny'], inplace = True)
print("\n 'MEB_KS' after drop of columns: \n")
print(data['MEB_KS'].head(5))

In [ ]:
#MEB_GROB 
# print(data['MEB_GROB'].columns)
# print(data['MEB_GROB'].head(5))
data['MEB_GROB'].drop(columns = ['shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested', 
                                'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'], inplace = True)
print("\n 'MEB_GROB' after drop of columns: \n")
print(data['MEB_GROB'].head(5))


<h1>Łączenie tabel</h1>

In [ ]:
#pd.set_option('display.max_columns', None)

# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)


#data['MEB_KO'].head()
#data['MEB_KO_DGM'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KO

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')

#  do wyrzucenia kolumna timestamp oraz eks

#data['MEB_DMC'].head(1)

In [ ]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM

data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
#data['MEB_DGM'].head(1)

In [ ]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 
# Tutaj będzie do zmiany, ponieważ usuwam część kolumn z ONI_CIRCTUIS - JR
# Przerobione ~ ŁO
oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['flow', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM

oni_circuits = oni_circuits.merge(data['MEB_DGM'], left_on='id_dmc', right_on='id', how='left')
#oni_circuits.head(1)

In [ ]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS

data['MEB_DMC'] = data['MEB_DMC'].merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left')
#data['MEB_DMC'].head(1)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
#data['MEB_DMC'].head(1)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
#data['MEB_DMC'].head(1)

In [ ]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

In [ ]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

<h1>Usuwanie anomalii</h1>

In [9]:
print(list(data.keys()))

['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS']


In [15]:
meb_dgm_df = data['MEB_DGM']

In [16]:
print(meb_dgm_df.head(5))

     id                    dmc  nr_dgm status  czas_fazy_1  czas_fazy_2  \
0   220  190627091611011003610    10.0      4       1850.0         88.0   
1  3416  190719071611011006077    10.0      1       2197.0         89.0   
2  3417  190719071811011006078    10.0      1       2184.0         86.0   
3  3418  190719072011011006079    10.0      1       2159.0         89.0   
4  3419  190719072111011006080    10.0      1       2175.0         89.0   

   czas_fazy_3  max_predkosc  cisnienie_tloka  cisnienie_koncowe  ...  \
0         47.5          6.52              7.0              264.0  ...   
1         46.5          6.59             11.0              285.0  ...   
2         45.0          6.52              8.0              284.0  ...   
3         45.5          6.54              6.0              284.0  ...   
4         47.0          6.57              8.0              283.0  ...   

   oni_temp_curr_f2  oni_temp_fore_f1  oni_temp_fore_f2  vds_air_pressure  \
0               NaN              

In [18]:
print(list(meb_dgm_df.keys()))

['id', 'dmc', 'nr_dgm', 'status', 'czas_fazy_1', 'czas_fazy_2', 'czas_fazy_3', 'max_predkosc', 'cisnienie_tloka', 'cisnienie_koncowe', 'nachdruck_hub', 'anguss', 'max_press_kolbenhub', 'temp_pieca', 'oni_temp_curr_f1', 'oni_temp_curr_f2', 'oni_temp_fore_f1', 'oni_temp_fore_f2', 'vds_air_pressure', 'vds_vac_hose1', 'vds_vac_hose2', 'vds_vac_tank', 'vds_vac_valve1', 'vds_vac_valve2', 'czas_taktu']


In [25]:
print(meb_dgm_df['nr_dgm'].unique())
print(len(meb_dgm_df))
print(type(meb_dgm_df['nr_dgm'][0]))
print(len(meb_dgm_df[meb_dgm_df['nr_dgm'] == 20]))



[10.  9.  4. nan  8. 16. 20.]
1357796
<class 'numpy.float64'>
3521
